# Voice Inpainting Debug Demo

In [1]:
from IPython.display import Audio, display
import torch
import time
from loguru import logger

from src.tokenization import AudioTokenizer
from src.semantic_edit import SemanticEditor
from src.integrated_inpainting import IntegratedVoiceInpainting
from src.main import setup_device

%load_ext autoreload
%autoreload 2

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



/Users/philipp/Documents/audio-inpaint/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


import error: No module named 'triton'


In [2]:
# Input parameters
input_file = "data/debug/2739d072-86c9-4606-9618-3f452716ebac/01_original.wav"
edit_prompt = "Change 'one piece' to 'two pieces'"
output_file = "output.wav"
debug_dir = "data/debug_output"

In [3]:
edits=edit_prompt

start_time = time.time()

# Set up device
device = setup_device()

# Step 1: Tokenize input audio to RVQ tokens
logger.info("Tokenizing input audio to RVQ tokens...")
tokenizer = AudioTokenizer(device=device)
tokenized_audio = tokenizer.tokenize(input_file)

# Step 2: Process the edits - either convert a single prompt to an edit operation
#         or use the provided list of edits
edit_operations = []

if isinstance(edits, str):
    # Single edit prompt provided - use SemanticEditor to find edit region
    logger.info(f"Processing single edit prompt: {edits}")
    editor = SemanticEditor(tokenizer, load_llm=True)
    edit_op = editor.find_edit_region(tokenized_audio, edits)
    edit_operations.append(edit_op)

2025-03-29 23:01:41.259 | WARNING  | src.main:setup_device:35 - MPS backend detected but not used due to compatibility issues with sparse tensors. Falling back to CPU. This will be slower but more reliable.
2025-03-29 23:01:41.259 | INFO     | src.main:setup_device:41 - Using device: cpu
2025-03-29 23:01:41.259 | INFO     | __main__:<module>:9 - Tokenizing input audio to RVQ tokens...
2025-03-29 23:01:41.260 | INFO     | src.tokenization:_initialize_tokenizers:71 - Initializing Mimi RVQ tokenizer...
2025-03-29 23:01:41.372 | INFO     | src.mimi_tokenizer:_initialize_moshi_mlx:38 - Initializing moshi_mlx backend
2025-03-29 23:01:41.373 | INFO     | src.mimi_tokenizer:_initialize_moshi_mlx:41 - Downloading moshi_mlx tokenizer weights
2025-03-29 23:01:41.552 | INFO     | src.mimi_tokenizer:_initialize_moshi_mlx:47 - Creating Mimi MLX models with 32 codebooks
2025-03-29 23:01:41.583 | INFO     | src.mimi_tokenizer:_initialize_moshi_mlx:57 - Warming up MLX models
2025-03-29 23:01:41.746 | I

In [4]:
edit_op

EditOperation(original_text='one piece', edited_text='two pieces', start_token_idx=17, end_token_idx=27, confidence=1.0, prepadding_text='I would like to have', prepadding_start_token_idx=6, prepadding_end_token_idx=17, postpadding_text='of chocolate as well as a strawberry cake.', postpadding_start_token_idx=27, postpadding_end_token_idx=59)

In [ ]:
temperature=0.7
topk=25

# Step 3: Perform integrated inpainting
logger.info("Performing integrated voice inpainting...")
inpainting = IntegratedVoiceInpainting(device=device)

with torch.inference_mode():
    inpainted_tokens, final_audio, final_sr = inpainting.batch_inpaint(
        tokenized_audio,
        edit_operations,
        temperature=temperature,
        topk=topk
    )

    elapsed_time = time.time() - start_time
    logger.info(
        f"Voice inpainting completed successfully in {elapsed_time:.2f} seconds"
    )

display(Audio(final_audio, rate=final_sr))

2025-03-29 23:02:03.627 | INFO     | __main__:<module>:5 - Performing integrated voice inpainting...
2025-03-29 23:02:03.628 | INFO     | src.integrated_inpainting:_initialize_csm_model:49 - Initializing CSM model...
2025-03-29 23:02:22.031 | INFO     | src.mimi_tokenizer:_initialize_moshi_mlx:38 - Initializing moshi_mlx backend
2025-03-29 23:02:22.036 | INFO     | src.mimi_tokenizer:_initialize_moshi_mlx:41 - Downloading moshi_mlx tokenizer weights
2025-03-29 23:02:22.161 | INFO     | src.mimi_tokenizer:_initialize_moshi_mlx:47 - Creating Mimi MLX models with 32 codebooks
2025-03-29 23:02:22.186 | INFO     | src.mimi_tokenizer:_initialize_moshi_mlx:57 - Warming up MLX models
2025-03-29 23:02:25.136 | INFO     | src.mimi_tokenizer:_initialize_moshi_mlx:61 - moshi_mlx initialization complete


ckpt path or config path does not exist! Downloading the model from the Hugging Face Hub...


Fetching 13 files: 100%|██████████| 13/13 [00:00<00:00, 199728.76it/s]
2025-03-29 23:02:26.176 | INFO     | src.integrated_inpainting:_initialize_csm_model:54 - CSM model loaded successfully
2025-03-29 23:02:26.176 | INFO     | src.tokenization:_initialize_tokenizers:71 - Initializing Mimi RVQ tokenizer...
2025-03-29 23:02:26.177 | INFO     | src.mimi_tokenizer:_initialize_moshi_mlx:38 - Initializing moshi_mlx backend
2025-03-29 23:02:26.177 | INFO     | src.mimi_tokenizer:_initialize_moshi_mlx:41 - Downloading moshi_mlx tokenizer weights
2025-03-29 23:02:26.334 | INFO     | src.mimi_tokenizer:_initialize_moshi_mlx:47 - Creating Mimi MLX models with 32 codebooks
2025-03-29 23:02:26.370 | INFO     | src.mimi_tokenizer:_initialize_moshi_mlx:57 - Warming up MLX models
2025-03-29 23:02:26.518 | INFO     | src.mimi_tokenizer:_initialize_moshi_mlx:61 - moshi_mlx initialization complete
2025-03-29 23:02:26.519 | INFO     | src.tokenization:_initialize_tokenizers:76 - Initializing Llama text t